In [89]:
%load_ext autoreload
%autoreload 2
import TRITON_SWMM_toolkit.examples as examples
import TRITON_SWMM_toolkit.experiment as experiment
from TRITON_SWMM_toolkit.processing_simulation import TRITONSWMM_sim_post_processing
import TRITON_SWMM_toolkit.processing_simulation as ps
import TRITON_SWMM_toolkit.processing_experiment as pe
from TRITON_SWMM_toolkit.processing_simulation import *
from importlib import reload
reload(examples)
reload(experiment)
reload(ps)
reload(pe)
norfolk_singlesim = examples.TRITON_SWMM_testcases.retreive_norfolk_single_sim_test_case(download_if_exists=False)
norfolk_multisim = examples.TRITON_SWMM_testcases.retreive_norfolk_multi_sim_test_case()

sim_iloc = 0
exp = norfolk_multisim.system.experiment
# scen = exp.scenarios[sim_iloc]
# run = exp._retreive_sim_run_object(0)
# proc = exp._retrieve_sim_run_processing_object(0)
self = exp.process

# proc.write_timeseries_outputs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
exp.log.print()

{
    "logfile": "/home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/norfolk_tests/multi_sim/log.json",
    "all_scenarios_created": true,
    "TRITONSWMM_compiled_successfully": true,
    "all_sims_run": true,
    "all_TRITON_timeseries_processed": true,
    "all_SWMM_timeseries_processed": true,
    "all_raw_TRITON_outputs_cleared": true,
    "all_raw_SWMM_outputs_cleared": true,
    "TRITON_experiment_summary_created": true,
    "SWMM_node_experiment_summary_created": null,
    "SWMM_link_experiment_summary_created": null,
    "processing_log": {
        "outputs": {
            "TRITON.zarr": {
                "filepath": "/home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/norfolk_tests/multi_sim/TRITON.zarr",
                "size_MiB": 0.05008983612060547,
                "time_elapsed_s": 1.8846004009246826,
                "success": true,
                "notes": "",
                "warnings

In [ ]:
max_mem_usage_MiB = 200
xy_chunksize = 256

lst_ds = []
for sim_iloc in self._experiment.df_sims.index:
    ds = exp._sim_run_processing_objects[sim_iloc].TRITON_timeseries
    ds = pe.summarize_triton_simulation_results(ds)
    ds = ds.assign_coords(coords = dict(sim_iloc = sim_iloc))
    ds = ds.expand_dims("sim_iloc")
    lst_ds.append(ds)


ds_triton_outputs = xr.concat(lst_ds, dim = "sim_iloc", combine_attrs="drop_conflicts")
memory_size_gb = ds_triton_outputs.nbytes / 1024 / 1024 / 1024


ds_triton_outputs = ds_triton_outputs.chunk(dict(sim_iloc = 4, x=xy_chunksize, y=xy_chunksize))
size_to_load_MiB = pe.ds_memory_req_MiB(ds_triton_outputs)
size_per_sim = size_to_load_MiB / len(lst_ds)
sim_idx_chunk = pe.prev_power_of_two(max_mem_usage_MiB / size_per_sim)

# ds_triton_outputs
size_to_load_MiB = pe.ds_memory_req_MiB(ds_triton_outputs.isel(sim_iloc = slice(0, sim_idx_chunk), x = slice(0, xy_chunksize), y= slice(0, xy_chunksize)))
size_to_load_MiB
# size_to_load_MiB = pe.ds_memory_req_MiB(ds_triton_outputs)




# ds_triton_outputs.isel(x = slice(0, 100), y= slice(0, 100))


10.503929138183594